In [20]:
# Import necessary libraries, install them if needed
# For compiling, you will need Jupyter pip package installed.

from sample_data import price_lists
from typing import Dict, Tuple, Optional
from functools import lru_cache
import json
import tests_dataset

This function cleans the price lists by removing any corrupt data entries, such as invalid prices or prefixes. It ensures that only valid data is processed in subsequent steps.

In [21]:
def clean_price_lists(price_lists: Dict[str, Dict[str, float]]) -> Dict[str, Dict[str, float]]:
    """
    Clean the price lists by removing any corrupt data entries.
    
    Args:
    price_lists (dict): A dictionary containing the price lists of operators.
    
    Returns:
    dict: A cleaned dictionary with valid price entries.
    """
    cleaned_lists = {}
    for operator, prices in price_lists.items():
        cleaned_prices = {}
        for prefix, price in prices.items():
            if isinstance(price, (int, float)) and price >= 0 and prefix.replace('.', '', 1).isdigit():
                cleaned_prices[prefix] = price
            else:
                print(f"Removed invalid entry from {operator}: prefix={prefix}, price={price}")
        cleaned_lists[operator] = cleaned_prices
    return cleaned_lists

This function preprocesses the cleaned price lists to allow for faster lookups. It ensures that only valid prefixes and prices are included in the preprocessed data.

In [22]:
def preprocess_price_lists(price_lists: Dict[str, Dict[str, float]]) -> Dict[str, Dict[str, float]]:
    """
    Preprocess the price lists to allow for faster lookups.

    Args:
    price_lists (dict): A dictionary containing the price lists of operators.

    Returns:
    dict: A dictionary where keys are operators and values are dictionaries of prefixes and prices.
    """
    preprocessed = {}
    for operator, prices in price_lists.items():
        preprocessed[operator] = {prefix: price for prefix, price in prices.items() if isinstance(price, (int, float)) and price >= 0}
    return preprocessed

In [23]:
def dict_to_json_str(d: Dict) -> str:
    """
    Convert a dictionary to a JSON string.
    
    Args:
    d (dict): The dictionary to convert.
    
    Returns:
    str: The JSON string representation of the dictionary.
    """
    return json.dumps(d, sort_keys=True)

This function finds the cheapest operator for a given phone number by comparing the prices from different operators. It uses caching (lru_cache) to improve performance by storing the results of expensive function calls. This is especially handy if we call the function over and over and will outperform other algorytms even if there is room for logic improvements of the insides.

In [24]:
@lru_cache(maxsize=None)
def find_cheapest_operator(phone_number: str, preprocessed_lists: str) -> Tuple[Optional[str], float]:
    """
    Find the cheapest operator for a given phone number based on the preprocessed price lists.

    Args:
    phone_number (str): The phone number to find the cheapest operator for.
    preprocessed_lists (str): A JSON string representation of the preprocessed price lists.

    Returns:
    tuple: The name of the cheapest operator and the price per minute for the given phone number.
    """
    preprocessed_lists = json.loads(preprocessed_lists)
    best_matches = {}

    # Find the longest matching prefix for each operator
    for operator, prices in preprocessed_lists.items():
        longest_prefix = ""
        best_price = float('inf')
        for prefix, price in prices.items():
            if phone_number.startswith(prefix):
                if len(prefix) > len(longest_prefix) or (len(prefix) == len(longest_prefix) and price < best_price):
                    longest_prefix = prefix
                    best_price = price
        if longest_prefix:
            best_matches[operator] = (longest_prefix, best_price)

    # Compare the best matches to find the cheapest option
    cheapest_operator = None
    cheapest_price = float('inf')
    longest_prefix_length = 0

    for operator, (prefix, price) in best_matches.items():
        if price < cheapest_price or (price == cheapest_price and len(prefix) > longest_prefix_length):
            cheapest_operator = operator
            cheapest_price = price
            longest_prefix_length = len(prefix)

    return cheapest_operator, cheapest_price


In [25]:
# Prepare data once for reuse

cleaned_lists = clean_price_lists(price_lists)
preprocessed_lists = preprocess_price_lists(cleaned_lists)
preprocessed_lists_json_str = dict_to_json_str(preprocessed_lists)

Removed invalid entry from Operator A: prefix=-1, price=-0.5
Removed invalid entry from Operator A: prefix=nan, price=nan
Removed invalid entry from Operator B: prefix=-44, price=-0.3
Removed invalid entry from Operator B: prefix=nan, price=nan


This function runs a series of tests to ensure the routing and price lookup functionality works correctly. It uses sample data, cleans and preprocesses it, and checks the results against expected values to validate the logic.

In [26]:
def run_tests():
    """
    Run a series of tests to ensure the routing and price lookup functionality works correctly.
    """
    # Use the test dataset from tests_dataset.py
    price_lists = tests_dataset.price_lists

    # Clean and preprocess the price lists
    cleaned_lists = clean_price_lists(price_lists)
    preprocessed_lists = preprocess_price_lists(cleaned_lists)

    # Convert to JSON for testing purposes
    preprocessed_lists_json_str = json.dumps(preprocessed_lists)

    # Use the test cases from tests_dataset.py
    tests = tests_dataset.tests

    for test in tests:
        operator, price = find_cheapest_operator(test['phone_number'], preprocessed_lists_json_str)
        assert operator in test['expected_operators'], f"Test failed for {test['phone_number']}: expected {test['expected_operators']}, got {operator}"
        assert price == test['expected_price'], f"Test failed for {test['phone_number']}: expected {test['expected_price']}, got {price}"
    print("All tests passed!")


In [27]:
if __name__ == "__main__":
    run_tests()
    phone_number = input("Enter the phone number: ")
    operator, price = find_cheapest_operator(phone_number, preprocessed_lists_json_str)
    if operator:
        print(f"The cheapest operator is {operator} with a price of ${price}/min.")
    else:
        print("No operator available for this phone number.")

AssertionError: Test failed for 4681234567: expected ['Operator A'], got Operator B

In [ ]:
# Code below is responsible for recompiling the compiled Py code.

# !jupyter nbconvert --to script telephone_routing.ipynb